# Data wrangling Pandas

## Learning outcomes
- Inspect a dataframe with `df.head()`, `df.tail()`, `df.info()`, `df.describe()`.
- Obtain dataframe summaries with `df.info()` and `df.describe()`.
- Rename columns of a dataframe using the `df.rename()` function or by accessing the `df.columns` attribute.
- Use `df.melt()` and `df.pivot()` to reshape dataframes, specifically to make tidy dataframes.
- Combine dataframes using `df.merge()` and `pd.concat()` and know when to use these different methods.
- Apply functions to a dataframe `df.apply()` and `df.applymap()`
- Perform grouping and aggregating operations using `df.groupby()` and `df.agg()`.
- Perform aggregating methods on grouped or ungrouped objects such as finding the minimum, maximum and sum of values in a dataframe using `df.agg()`.
- Remove or fill missing values in a dataframe with `df.dropna()` and `df.fillna()`.

In [ ]:
import numpy as np
import pandas as pd

## DataFrame characteristics
---

- Last lecture we looked at how we can create dataframes
- Let's now look at some helpful ways we can view our dataframe

### Head/Tail

- The `.head()` and `.tail()` methods allow you to view the top/bottom *n* (default 5) rows of a dataframe
-  [IMDB movie dataset](https://www.kaggle.com/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows)
- Instead of using the one on Kaggle we will use a local dataset. Go to EdStem to download the [zip folder](https://edstem.org/us/courses/31933/resources?download=23687) that contains the data folder for the labs. Place the data folder in the same directory that your Jupyter notebook is in Let’s first load the dataset which is in a csv file.

In [ ]:
df = pd.read_csv('data/imdb.csv')
df

- The default return value is 5 rows, but we can pass in any number we like
- For example, let's take a look at the top 10 rows:

In [ ]:
df.head(10)

- Or the bottom 5 rows:

In [ ]:
df.tail()

### DataFrame summaries

- Three very helpful attributes/functions for getting high-level summaries of your dataframe are:
    - `.shape`
    - `.info()`
    - `.describe()`

- `.shape` is just like the ndarray attribute we've seen previously
- It gives the shape (rows, cols) of your dataframe:

In [ ]:
df.shape

- `.info()` prints information about the dataframe itself, such as dtypes, memory usages, non-null values, etc:

In [ ]:
df.info()

- `.describe()` provides summary statistics of the values within a dataframe:

In [ ]:
df.describe()

- By default, `.describe()` only print summaries of numeric features
- We can force it to give summaries on all features using the argument `include='all'` (although they may not make sense!):

In [ ]:
df.describe(include='all')

### Displaying DataFrames

- Displaying your dataframes effectively can be an important part of your workflow
- If a dataframe has more than 60 rows, Pandas will only display the first 5 and last 5 rows:

In [ ]:
pd.DataFrame(np.random.rand(100))

- For dataframes of less than 60 rows, Pandas will print the whole dataframe

In [ ]:
pd.DataFrame(np.random.rand(25))

### Views vs copies

- In previous lectures we've discussed views ("looking" at a part of an existing object) and copies (making a new copy of the object in memory)
- These things get a little abstract with Pandas and "...it’s very hard to predict whether it will return a view or a copy" (that's a quote straight [from a dedicated section in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy))
- Basically, it depends on the operation you are trying to perform, your dataframe's structure and the memory layout of the underlying array
- But don't worry, let me tell you all you need to know
- Firstly, the most common warning you'll encounter in Pandas is the `SettingWithCopy`, Pandas raises it as a warning that you might not be doing what you think you're doing
- Let's see an example: one of movies in our dataframe has an incorrect value in the `Released_Year` field:

In [ ]:
df[df['Released_Year'] > 2021]

- Imagine we wanted to change this to `2010`
- You'd probably do the following:

In [ ]:
df[df['Released_Year'] > 2021]['Released_Year'] = 2010

- Ah, there's that warning
- Did our dataframe get changed?

In [ ]:
df[df['Released_Year'] > 2021]

- No it didn't, even though you probably thought it did
- What happened above is that `df[df['Released_Year'] > 2021]` was executed first and returned a copy of the dataframe, we can confirm by using `id()`:

In [ ]:
print(f"The id of the original dataframe is: {id(df)}")
print(f" The id of the indexed dataframe is: {id(df[df['Released_Year'] > 2021])}")

- We then tried to set a value on this new object by appending `['Released_Year'] = 2010`
- Pandas is warning us that we are doing that operation on a copy of the original dataframe, which is probably not what we want
- To fix this, you need to index in a single go, using `.loc[]` for example:

In [ ]:
df.loc[df['Released_Year'] > 2021, 'Released_Year'] = 2010

- No error this time! And let's confirm the change:

In [ ]:
df[df['Released_Year'] > 2021]

- The second thing you need to know is that if you're ever in doubt about whether something is a view or a copy, you can just use the `.copy()` method to force a copy of a dataframe
- Just like this:

In [ ]:
df2 = df[df['Released_Year'] > 2021].copy()

- That way, your guaranteed a copy that you can modify as you wish

## Basic DataFrame manipulations
---

### Renaming columns

- We can rename columns two ways:
    1. Using `.rename()` (to selectively change column names)
    2. By setting the `.columns` attribute (to change all column names at once)

In [ ]:
df

- Let's give it a go:

In [ ]:
df.rename(columns={"Released_Year": "Year",
                   "Overview": "Synopsis"})
df

- Wait? What happened? Nothing changed?
- In the code above we did actually rename columns of our dataframe but we didn't modify the dataframe inplace, we made a copy of it (more on that later)
- There are generally two options for making permanent dataframe changes:
    - 1. Use the argument `inplace=True`, e.g., `df.rename(..., inplace=True)`, available in most functions/methods
    - 2. Re-assign, e.g., `df = df.rename(...)`
- The Pandas team recommends **Method 2 (re-assign)**, for a [few reasons](https://www.youtube.com/watch?v=hK6o_TDXXN8&t=700) (mostly to do with how memory is allocated under the hood)

In [ ]:
df = df.rename(columns={"Released_Year": "Year",
                   "Overview": "Synopsis"})
df

- If you wish to change all of the columns of a dataframe, you can do so by setting the `.columns` attribute

In [ ]:
df.columns = [f"Column {_}" for _ in range(15)]
df

- You can change the index labels of a dataframe in 3 main ways:
    1. `.set_index()` to make one of the columns of the dataframe the index
    2. Directly modify `df.index.name` to change the index name
    3. `.reset_index()` to move the current index as a column and to reset the index with integer labels starting from 0
    4. Directly modify the `.index()` attribute

In [ ]:
df

### Adding/Removing columns

- There are two main ways to add/remove columns of a dataframe
    1. Use `[]` to add columns
    2. Use `.drop()` to drop columns
- Let's re-read in a fresh copy of IMDB movie dataset:

In [ ]:
df = pd.read_csv('data/imdb.csv')
df

- We can add a new column to a dataframe by simply using `[]` with a new column name and value(s)

In [ ]:
df['RottenTomato_score'] = 0

In [ ]:
df = df.drop(columns=['Star3', 'Star4'])
df

### Adding/Removing rows

- You won't often be adding rows to a dataframe manually (you'll usually add rows through concatenating/joining - that's coming up next)
- You can add/remove rows of a dataframe in two ways:
    1. Use `.append()` to add rows
    2. Use `.drop()` to drop rows

In [ ]:
df

- Let's add a new row to the bottom of this dataframe

In [ ]:
another_row = pd.DataFrame(
    [
        [
            "Zone 414",
            "2021",
            "R",
            "98 min",
            "Sci-Fi",
            "6.5",
            "Set in the near future in a colony of state-of-the-art humanoid robots.",
            75.0,
            "Andrew Baird",
            "Travis Fimmel",
            "Guy Pearce",
            12343,
            1229123,
            0
        ]
    ],
    columns=df.columns,
    index=[1100]
)
df = df.append(another_row)
df

- We can drop all rows beyond row 5 using `.drop()`

In [ ]:
df.drop(df.index[5:], axis=0)

We can also drop rows indirectly by slicing the original dataframe and assign the result to a new one:

In [ ]:
df2 = df.iloc[:5, :]
df2

## DataFrame reshaping
---

### Tidy data

- [Tidy data](https://vita.had.co.nz/papers/tidy-data.pdf) is about "linking the structure of a dataset with its semantics (its meaning)"
- You've already looked at tidy data in your previous courses
- It is defined by:
    1. Each variable forms a column
    2. Each observation forms a row
    3. Each type of observational unit forms a table
- Often you'll need to reshape a dataframe to make it tidy (or for some other purpose)
    
![](img/lecture7/tidy.png)

Source: [r4ds](https://r4ds.had.co.nz/tidy-data.html#fig:tidy-structure)

### Melt and pivot

- Pandas `.melt()`, `.pivot()` and `.pivot_table()` can help reshape dataframes
    - `.melt()`: make wide data long (like `melt()` in R)
    - `.pivot()`: make long data width (like `cast()` in R)
    - `.pivot_table()`: same as `.pivot()` but can handle multiple indexes
    
![](img/lecture7/melt_pivot.gif)

Source: [Garrick Aden-Buie's GitHub](https://github.com/gadenbuie/tidyexplain#spread-and-gather)

- The below data shows how many courses different instructors taught across different years:

In [ ]:
df = pd.DataFrame({"Name": ["Arman", "Mike", "Tiffany", "Varada", "Joel"],
                   "2018": [1, 3, 4, 5, 3],
                   "2019": [2, 4, 3, 2, 1],
                   "2020": [5, 2, 4, 4, 3]})
df

- Let's try to tidy the data with `.melt()`. We first have to know what exactly `.melt()` does, so let's apply it to our dataframe without any arguments:

In [ ]:
df.melt()

- Think of `.melt()` as trying to make everything look like `key: value` pairs
- By default, `.melt()` takes each column name as a key and binds it with all column values
- Here we're interested in questions about each instructor, so we want the rows to be identified with instructor names
- This can be done using the `id_vars` argument, which determines which column should be the "identifier", i.e. the "key":

In [ ]:
df.melt(id_vars="Name")

Much better!

- The `value_vars` argument allows us to select which specific variables we want to "melt" (if you don't specify `value_vars`, all non-identifier columns will be used)
- For example, below I'm only using the `2020` column:

In [ ]:
df.melt(id_vars="Name", value_vars=["2020"])

In [ ]:
df_melt = df.melt(id_vars="Name", var_name="Year")
df_melt

- The `value` column can be renamed by passing a name to the `value_name` argument:

In [ ]:
df_melt = df.melt(id_vars="Name", var_name="Year", value_name="Course")
df_melt

- Sometimes, you want to make long data wide, which we can do with `.pivot()`
- When using `.pivot()` we need to specify the `index` to pivot on, and the `columns` that will be used to make the new columns of the wider dataframe:

In [ ]:
df_pivot = df_melt.pivot(index="Name",
                         columns="Year",
                         values="Course"
                        )
df_pivot

- You'll notice that Pandas set our specified `index="Name"` argument as the index of the new dataframe, and preserved the label of the columns
- We can easily remove these names and reset the index to make our dataframe look like it originally did

In [ ]:
df_pivot.columns

In [ ]:
df_pivot.columns.name = None
df_pivot = df_pivot.reset_index()
df_pivot

- `.pivot()` will often get you what you want, but it won't work if you want to:
    - Use multiple indexes (next lecture)
    - Have duplicate index/column labels
- In these cases you'll have to use `.pivot_table()`
- I won't focus on it too much here because I'd rather you learn about `pivot()` first

In [ ]:
df = pd.DataFrame(
    {
        "Name": ["Arman", "Arman", "Mike", "Mike"],
        "Department": ["CS", "STATS", "CS", "STATS"],
        "2018": [1, 2, 3, 1],
        "2019": [2, 3, 4, 2],
        "2020": [5, 1, 2, 2],
    }
)
df

In [ ]:
df = df.melt(
    id_vars=["Name", "Department"],
    var_name="Year",
    value_name="Courses"
)
df

- In the above case, we have duplicates in `Name`, so `pivot()` won't work
- It will throw us a `ValueError: Index contains duplicate entries, cannot reshape`:

In [ ]:
df.pivot(index="Name",
         columns="Year",
         values="Courses")

- In such a case, we'd use `.pivot_table()`
- It will apply an aggregation function to our duplicates, in this case, we'll `sum()` them up:

In [ ]:
df.pivot_table(index="Name", columns='Year', values='Courses', aggfunc='sum')

- If we wanted to keep the numbers per department, we could specify both `Name` and `Department` as multiple indexes:

In [ ]:
df.pivot_table(index=["Name", "Department"], columns='Year', values='Courses')

- The result above is a mutlti-index or "hierarchically indexed" dataframe (more on those next lecture)
- If you ever have a need to use it, you can read more about `pivot_table()` in the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#pivot-tables)

## More DataFrame operations
---

### Applying custom functions

- There will be times when you want to apply a function that is not built-in to Pandas
- For this, we also have methods:
    - `df.apply()`, applies a function column-wise or row-wise across a dataframe (the function must be able to accept/return an array)
    - `df.applymap()`, applies a function element-wise (for functions that accept/return single values at a time)
    - `series.apply()`/`series.map()`, same as above but for Pandas series

- For example, say you want to use a numpy function on a column in your dataframe

In [ ]:
df = pd.read_csv('data/YVR_weather_data.csv', usecols=range(0, 4))
df[['Mean Max Temp (°C)']].apply(np.log)

- Or you may want to apply your own custom function

In [ ]:
def convert_to_fahrenheit(x):
    return x * (9 / 5) + 32


df[['Mean Max Temp (°C)']].apply(convert_to_fahrenheit)

- This may have been better as a lambda function...

In [ ]:
df[['Mean Max Temp (°C)']].apply(lambda x: x * (9 / 5) + 32)

- You can even use functions that require additional arguments
- Just specify the arguments in `.apply()`

In [ ]:
def convert_temperature(x, to="absolute"):
    if to == "absolute":
        return x + 273.15
    elif to == "fahrenheit":
        return x * (9 / 5) + 32


df[['Mean Max Temp (°C)']].apply(convert_temperature, to="absolute")

### Grouping

- Often we are interested in examining specific groups in our data
- `df.groupby()` allows us to group our data based on a variable(s)
- Analgous to the `group_by` function in R

In [ ]:
df = pd.read_csv('data/imdb.csv')
df

- Let's group this dataframe on the column `Name`

In [ ]:
dfg = df.groupby(by='Genre')
dfg

- What is a `DataFrameGroupBy` object?
- It contains information about the groups of the dataframe

- The groupby object is really just a dictionary of index-mappings, which we could look at if we wanted to

In [ ]:
dfg.groups

- We can also access a group using the `.get_group()` method

In [ ]:
dfg.get_group('Action, Adventure')

- The usual thing to do, however, is to apply aggregation functions to the groupby object

In [ ]:
dfg.mean()[['IMDB_Rating']]

- We can apply multiple functions using `.aggregate()`

In [ ]:
dfg.aggregate(['mean', 'sum', 'count'])

- And even apply different functions to different columns

In [ ]:
def num_range(x):
    return x.max() - x.min()

dfg.aggregate({"Meta_score": ['max', 'min', 'mean', num_range], 
               "Gross": ['sum']})

- By the way, you can use aggregate for non-grouped dataframes too
- This is pretty much what `df.describe` does under the hood

In [ ]:
df.agg(['mean', 'min', 'count', num_range])

### Dealing with missing values

- Missing values are typically denoted with `NaN`
- We can use `df.isna()` or `df.isnull()` to find missing values in a dataframe (both functions do exactly the same thing!)
- It returns a boolean for each element in the dataframe

In [ ]:
df.isna()

- But it's usually more helpful to get this information by row or by column using the `.any()` or `.info()` method

In [ ]:
df.info()

In [ ]:
df[df.isnull().any(axis=1)]

- When you have missing values, we usually either drop them or impute them
- You'll learn more about imputing in DSCI 562/571/573
- For now, you can drop missing values using `df.dropna()`

In [ ]:
df.dropna()

- Or you can fill them using `.fillna()`
- This method has various options for filling, you can use a fixed value, the mean of the column, the previous non-nan value, etc
- You'll use this method more in the machine learning courses

In [ ]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns=list('ABCD'))
df

In [ ]:
df.fillna(0)  # fill with 0

In [ ]:
df.fillna(df.mean())  # fill with the mean

In [ ]:
df.fillna(method='bfill')  # backward (upwards) propagate last valid value

In [ ]:
df.fillna(method='ffill')  # forward (downward) propagate last valid value

## Deliverables

Now that you have had a whirlwind introduction to Pandas, complete the [Finding Pandas quiz on Canvas](https://canvas.ubc.ca/courses/106515/quizzes/579039).
While this quiz is graded for participation and not performance, your performance is a strong indicator of your surface knowledge to use Pandas.
We will use Pandas regularly in this course, so going over these modules multiple times is beneficial.
